## Методические указания по выполнению лабораторной работы №5

**Тема: Обучение модели YOLO на кастомном датасете и исследование влияния гиперпараметров на качество детекции**

**Цель работы:** Познакомиться с архитектурой YOLO на примере проверки гипотезы о релевантной метрике.

**Задачи:**

- Ознакомиться с архитектурой YOLO.
- Изучить метрики для анализа производительности модели, выбрать целевую метрику в соответствии с вариантом.
- Выбрать предметную область, сформировать гипотезу для проведения исследования.
- Собрать и проаннотировать данные, сформировать датасет.
- Провести fine-tuning предобученной модели YOLOv11 Nano/Small.
- Визуализировать и проанализировать результаты.
- На основе анализа сделать корректировку гиперпараметров/данных и провести вторую итерацию для повышения показателей.

### 1. Подготовка к обучению

#### 1.1 Метрики

Вариант 1 - Precision

#### 1.2 Гипотеза

Гипотеза должна отражать характер уклона исследования с обоснованием, отталкиваясь от предметной области. *Одна лишь констатация необходимости достижения высокого значения целевой метрики не является обоснованием*.

#### 1.3 Данные

Соберите не менее 500 изображений из открытых источников. Можно пользоваться готовыми наборами данных, но важно проверить качество: разрешение изображений, качество аннотаций, баланс классов. При самостоятельном сборе данных можете воспользоваться терминальной утилитой ffmpeg для нарезки видео на кадры и любым удобным инструментом аннотирования (Roboflow, CVAT и тд). 

#### 1.4 Предобработка

Примените методы аугментации к данным для расширения объема датасета для получения 1.5-2к изображений. Подготовьте данные к требуемому формату для обучающего процесса.

### 2. Обучение модели

#### 2.1 Подготовка окружения

Установите зависимости и библиотеки:

In [10]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from ultralytics import YOLO
import torch
from contextlib import redirect_stdout

#### 2.2 Подготовка модели

Загрузите предобученную модель, определите устройство, переведите модель в режим инференса. Не используйте размер модели больше чем Small для достижения лучших показателей на стандартных гиперпараметрах (особенно imgsz)

In [11]:
# импорт модели
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

model = YOLO("yolo11_I.pt", verbose=False).to(device)

Using device: cuda


In [12]:
import torch
print(torch.__version__)
print(torch.version.cuda)  # Проверка версии CUDA
print(torch.cuda.is_available())  # Проверка доступности GPU

2.6.0+cu126
12.6
True


#### 2.3 Загрузка и предобработка изображений


Затем импортируйте датасет в проект и выполните трансформацию данных (при использовании Roboflow трансформация выполняется на этапе предобработки):

In [13]:
# загрузка датасета
data_path = "C:/programming_HUB/MTUCI_projects/6_sem/neural_networks____university-homework/5_lab/dataset/dataset.yaml"
global_epochs = 10


#### 2.4 Обучение, оценка модели и визуализация результатов

Проведите обучение модели, проанализируйте кривые обучения, метрики и тестовые данные. Сделайте вывод и корректироваки для достижения лучших показателей

In [14]:
# обучение, оценка, визуализация
%matplotlib inline

params = {
    'data': data_path,
    'epochs': global_epochs,
    'imgsz': 640,
    'batch': 16,
    'device': device,
    'workers': 4,
    'optimizer': 'AdamW',
    'seed': 42
}
# Обучение модели

results = model.train(**params)

# Оценка модели на тестовых данных
metrics = model.val()
model.save('yolo11_I.pt')

# Визуализация результатов обучения

# Получаем историю обучения из CSV файла
# Путь к файлу с результатами
results_csv_path = os.path.join(model.trainer.save_dir, 'results.csv')

# Загружаем данные
try:
    results_df = pd.read_csv(results_csv_path)

    # Графики метрик
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(results_df['metrics/precision(B)'], label='Precision')
    plt.plot(results_df['metrics/recall(B)'], label='Recall')
    plt.plot(results_df['metrics/mAP50(B)'], label='mAP@0.5')
    plt.title('Метрики во время обучения')
    plt.xlabel('Эпоха')
    plt.ylabel('Значение')
    plt.legend()

    # Графики потерь
    plt.subplot(1, 2, 2)
    plt.plot(results_df['train/box_loss'], label='Train Box Loss')
    plt.plot(results_df['val/box_loss'], label='Val Box Loss')
    plt.title('Графики потерь')
    plt.xlabel('Эпоха')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()

except FileNotFoundError:
    print("Файл с результатами не найден")

New https://pypi.org/project/ultralytics/8.3.140 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.134  Python-3.12.4 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/programming_HUB/MTUCI_projects/6_sem/neural_networks____university-homework/5_lab/dataset/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentu

train: Scanning C:\programming_HUB\MTUCI_projects\6_sem\neural_networks____university-homework\5_lab\dataset\labels\train.cache... 355 images, 646 backgrounds, 0 corrupt: 100%|██████████| 1001/1001 [00:00<?, ?it/s]

KeyboardInterrupt



In [6]:
print(f"map: {metrics.box.map}")
plt.show()

map: 0.615027953213465


In [7]:
torch.cuda.empty_cache()
del model
import gc
gc.collect()

50491

#### 2.5 Вторая итерация

Проведите процедуры для достижения высоких показателей (корректировка данных/гиперпараметров), сделайте вывод


In [ ]:
%matplotlib inline
model = YOLO("yolo11_I.pt", verbose=False).to(device)

params_v2 = {
    'data': data_path,
    'epochs': global_epochs,
    'imgsz': 640,
    'batch': 16,
    'device': device,
    'workers': 4,
    'optimizer': 'AdamW',
    'seed': 42,

    # 'hsv_h': 0.3,
    # 'hsv_s': 0.7,
    # 'hsv_v': 0.4,
    # 'degrees': 15,
    # 'mixup': 0.2,
    #
    'lr0': 0.001,
    'lrf': 0.01,
    'weight_decay': 0.0005,
    # 'warmup_epochs': 3,
    # 'box': 7.5,
    # 'cls': 0.5,
    # 'dfl': 1.5,
}

# Повторное обучение модели с новыми параметрами
print("\nНачинаем вторую итерацию обучения с улучшенными параметрами")
results_v2 = model.train(**params_v2)

# Оценка новой модели
print("\nОценка улучшенной модели.")
metrics_v2 = model.val()

# Сохранение улучшенной модели
model.save('yolo11_II.pt')


In [14]:
if 'metrics_v2' not in locals():
    import torch
    from ultralytics import YOLO
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    model = YOLO('yolo11_II.pt').to(device)
    metrics_v2 = model.val()

if 'metrics' not in locals():
    import torch
    from ultralytics import YOLO
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
    model = YOLO('yolo11_I.pt').to(device)
    metrics = model.val()


def format_metric(metric):
    """Форматирует метрику, которая может быть numpy array или float"""
    if hasattr(metric, '__len__'):
        return f"{metric[0]:.4f}" if len(metric) > 0 else "N/A"
    return f"{metric:.4f}"

print("\nМетрики после второй итерации:")

print(f"Precision: {format_metric(metrics_v2.box.p)} (было {format_metric(metrics.box.p)}). Разница {(format_metric(metrics.box.p)/format_metric(metrics_v2.box.p))*100-100:.2f}%")

print(f"Recall: {format_metric(metrics_v2.box.r)} (было {format_metric(metrics.box.r)}). Разница {(format_metric(metrics.box.r)/format_metric(metrics_v2.box.r))*100-100:.2f}%")

print(f"mAP@0.5: {format_metric(metrics_v2.box.map50)} (было {format_metric(metrics.box.map50)}). Разница {(format_metric(metrics.box.map50)/format_metric(metrics_v2.box.map50))*100-100:.2f}%")

print(f"mAP@0.5:0.95: {format_metric(metrics_v2.box.map)} (было {format_metric(metrics.box.map)}). Разница {(format_metric(metrics.box.map)/format_metric(metrics_v2.box.map))*100-100:.2f}%")

# Визуализация результатов второй итерации
try:

    # Загрузка истории обучения
    results_csv_v2 = os.path.join(model.trainer.save_dir, 'results.csv')
    history_v2 = pd.read_csv(results_csv_v2)

    # Настройка графиков
    plt.figure(figsize=(15, 10))

    # График метрик
    plt.subplot(2, 2, 1)
    plt.plot(history_v2['epoch'], history_v2['metrics/precision(B)'], label='Precision', color='blue')
    plt.plot(history_v2['epoch'], history_v2['metrics/recall(B)'], label='Recall', color='green')
    plt.plot(history_v2['epoch'], history_v2['metrics/mAP50(B)'], label='mAP@0.5', color='red')
    plt.title('Гетрики метрик второй итерации')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.legend()
    plt.grid()

    # График потерь
    plt.subplot(2, 2, 2)
    plt.plot(history_v2['epoch'], history_v2['train/box_loss'], label='Train Box Loss', color='blue')
    plt.plot(history_v2['epoch'], history_v2['val/box_loss'], label='Val Box Loss', color='orange')
    plt.title('График потерь второй итерации')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()

    # Сравнение mAP до и после
    plt.subplot(2, 2, 3)
    plt.bar(['Первая версия', 'Вторая версия'],
            [metrics.box.map50, metrics_v2.box.map50],
            color=['blue', 'green'])
    plt.title('Сравнение mAP@0.5')
    plt.ylabel('mAP@0.5')
    plt.grid(axis='y')

    # Сравнение Precision-Recall
    plt.subplot(2, 2, 4)
    plt.scatter(metrics.box.p, metrics.box.r, color='blue', s=100, label='First Iteration')
    plt.scatter(metrics_v2.box.p, metrics_v2.box.r, color='green', s=100, label='Second Iteration')
    plt.xlabel('Precision')
    plt.ylabel('Recall')
    plt.title('Сравнение Precision-Recall')
    plt.legend()
    plt.grid()
    plt.xlim(left=0)
    plt.xlim(right=1)
    plt.ylim(bottom=0)
    plt.ylim(top=1)

    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"Ошибка при визуализации результатов второй итерации: {e}")

print("\nУлучшенная модель сохранена как 'yolov11n_improved.pt'")

YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs


FileNotFoundError: '/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml' does not exist